In [2]:
import requests as re
from dataclasses import dataclass, asdict
from time import sleep

Для начала соберём категории с сайта `https://www.kommersant.ru`. У него есть достаточно удобное API, с помощью которого можно без труда собрать новости

In [3]:
getNewsFormat = "https://www.kommersant.ru/listpage/lazyloaddocs?regionid=0&listtypeid=1&listid={}&date=&intervaltype=&idafter={}"

Для начала соберём рубрики (категории) с сайта 

Id рубрики в запросе устанавливаются параметром `listid`, в ответе будет получен `json` со списком нововстей. У каждой новости в поле `MainTag` будет название рубрики.
Таким образом перебирая все Id от 0 до 200 (200 выбрана как верхняя граница просто потому что), мы получим некоторую часть рубрик

In [5]:
rubrics_raw: dict[str, int] = {}
for rubricid in range(0, 200):
    resp = re.get(getNewsFormat.format(rubricid, 0))
    if resp.status_code != 200:
        continue

    data = resp.json()
    items = data["Items"]
    if len(items) == 0:
        continue
    rubric = items[0]['MainTagTitlle']
    rubrics_raw[rubric] = rubricid


In [6]:
rubrics_raw

{None: 0,
 'Политика': 2,
 'Экономика': 3,
 'Бизнес': 4,
 'Мир': 5,
 'Происшествия': 6,
 'Общество': 7,
 'Культура ': 8,
 'Спорт': 9,
 'Календарь недели': 13,
 'Новости': 57,
 'Фотогалерея': 18,
 'Книги': 19,
 'Видео': 110,
 'Вести 24': 24,
 'Forbes': 25,
 'Гороскоп': 26,
 'СФ - Банк решений': 27,
 'iOne конкурс': 28,
 'Вакансии': 29,
 'iOne темы': 30,
 'Аудио': 32,
 'Видео-тесты': 33,
 'Темы дня': 103,
 'Выпуски программ': 35,
 'Спецпроекты': 42,
 'Интервью': 125,
 'Финансы': 40,
 'Потребительский рынок': 41,
 'Онлайн-трансляции': 43,
 'Широкий документ': 44,
 'Широкий док с правой': 45,
 'Cover Story': 46,
 'Страна': 47,
 'Промышленность': 48,
 'Бизнес-форумы': 51,
 'Новости медиа форума': 52,
 'Media forum news': 53,
 'Интервью с ключевыми фигурами торговой политики': 55,
 'Аналитические материалы': 56,
 'Interviews with key figures of trade policy': 58,
 'Analytic materials': 59,
 'News': 60,
 'Фотогалереи': 64,
 'Photo galleries': 65,
 'Академия Журналистики': 66,
 'Авто': 68,
 'П

Выберем небольшую группу категорий

In [7]:
rubrics: dict[str, int] = {
    "Экономика": 3,
    "Политика": 2,
    "Бизнес": 4,
    "Мир": 5,
    "Происшествия": 6,
    "Общество": 7,
    "Культура ": 8,
    "Спорт": 9,
    "Финансы": 40,
    "Потребительский рынок": 41,
    "Телекоммуникации": 138,
    "Hi-Tech": 80,
}


In [8]:
@dataclass
class NewsInfo:
    doc_id: int
    tags: list[str]
    category: str
    title: str
    text: str


In [9]:
corpus: dict[str, dict[int, NewsInfo]] = {rubric: {} for rubric in rubrics}

In [10]:
for rubric, rubricid in rubrics.items():
    collected_news: dict[int, NewsInfo] = corpus[rubric]
    rubric_news_count = 1000
    idafter = 0
    resp_times = []

    while (len(corpus[rubric]) <= rubric_news_count):
        url = getNewsFormat.format(rubricid, idafter)
        resp = re.get(url)
        if resp.status_code != 200:
            print(url)
            continue
        resp_news = resp.json()["Items"] 
        resp_times.append(resp.elapsed)
        for news in resp_news:
            doc_id = news["DocsID"]
            title = news["Title"]
            tags = news["Tags"]

            selected_tags = []
            for tag in tags:
                if tag["Type"] in [1, 3]: # 1 - rubric, 3 - theme
                    selected_tags.append(tag["Name"])


            collected_news[doc_id] = NewsInfo(doc_id, selected_tags, rubric, title, "")

        idafter = min([it["DocsID"] for it in resp_news ])

        time_to_finish = ((rubric_news_count - len(collected_news)) // 20) * sum([t.total_seconds() for t in resp_times]) / len(resp_times)
        if len(resp_news) == 0:
            break
        print(f"Категория: '{rubric}'\tСобрано: {len(collected_news):4d}\t\
                Собрано%: {len(collected_news)/ rubric_news_count * 100:06.2f}%\t\
                Время зароса: {resp.elapsed.total_seconds():.3f}\tВремени осталось:{time_to_finish:.3f}") 
    corpus[rubric] = collected_news
    


Категория: 'Экономика'	Собрано:   20	                Собрано%: 002.00%	                Время зароса: 0.084	Времени осталось:4.128
Категория: 'Экономика'	Собрано:   40	                Собрано%: 004.00%	                Время зароса: 0.167	Времени осталось:6.028
Категория: 'Экономика'	Собрано:   60	                Собрано%: 006.00%	                Время зароса: 0.076	Времени осталось:5.129
Категория: 'Экономика'	Собрано:   80	                Собрано%: 008.00%	                Время зароса: 0.084	Времени осталось:4.728
Категория: 'Экономика'	Собрано:   99	                Собрано%: 009.90%	                Время зароса: 0.100	Времени осталось:4.604
Категория: 'Экономика'	Собрано:  108	                Собрано%: 010.80%	                Время зароса: 0.089	Времени осталось:4.407
Категория: 'Экономика'	Собрано:  128	                Собрано%: 012.80%	                Время зароса: 0.055	Времени осталось:4.031
Категория: 'Экономика'	Собрано:  148	                Собрано%: 014.80%	                Вре

In [13]:
corpus_flat: dict[int, NewsInfo] = {}
for news in corpus.values():
    for id, n in news.items():
        n.text = ""
        corpus_flat[id] = n

In [14]:
getDocTextFormat = "https://www.kommersant.ru/doc/{}"

In [15]:
from bs4 import BeautifulSoup

In [16]:
downloaded = 0
resp_times = []
for id, info in corpus_flat.items():
    url = getDocTextFormat.format(id)
    resp = re.get(url)

    if resp.status_code != 200:
        print(resp.status_code, id)
        continue
    if len(info.text) != 0:
        continue

    resp_times.append(resp.elapsed.total_seconds())
    downloaded +=1
    soup = BeautifulSoup(resp.content, 'html.parser')
    text = ""
    for p in soup.find("div", class_="lenta_top_doc").find_all("p", class_="doc__text"):
        if "document_authors" in p.attrs["class"]:
            continue
        text += p.get_text()
    info.text = text


    estimated_time = (len(corpus_flat) - downloaded) * sum(resp_times) / len(resp_times)
    if downloaded % 100 == 0:
        print(f"{downloaded}:{len(corpus_flat)}\t {downloaded/len(corpus_flat) * 100}\t Осталось: {estimated_time} c.")
     
    

100:12098	 0.8265829062654985	 Осталось: 1668.21907714 c.
200:12098	 1.653165812530997	 Осталось: 1649.70547047 c.
300:12098	 2.479748718796495	 Осталось: 1658.37796534 c.
400:12098	 3.306331625061994	 Осталось: 1711.11354445 c.
500:12098	 4.132914531327493	 Осталось: 1680.6645098879999 c.
600:12098	 4.95949743759299	 Осталось: 1653.5810948233334 c.
700:12098	 5.786080343858489	 Осталось: 1632.2647723685714 c.
800:12098	 6.612663250123988	 Осталось: 1632.416244375 c.
900:12098	 7.439246156389485	 Осталось: 1620.6637831755554 c.
1000:12098	 8.265829062654985	 Осталось: 1620.1955994559999 c.
1100:12098	 9.092411968920484	 Осталось: 1605.6123174 c.
1200:12098	 9.91899487518598	 Осталось: 1592.0853144766666 c.
1300:12098	 10.745577781451479	 Осталось: 1585.6791982384616 c.
1400:12098	 11.572160687716979	 Осталось: 1578.6918400328573 c.
1500:12098	 12.398743593982477	 Осталось: 1563.50089616 c.
1600:12098	 13.225326500247975	 Осталось: 1549.57293202 c.
1700:12098	 14.051909406513474	 Остало

In [25]:
data = []
for id, n in corpus_flat.items():
    n_data = {
        "article_id": getDocTextFormat.format(n.doc_id),
        "title": n.title,
        "category": n.category,
        "tags": ",".join(n.tags),
        "text": n.text
    }
    data.append(n_data)

In [32]:
import json
with open("corpus_raw.json", "w", encoding="utf8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)